In [2]:
import pandas as pd
import numpy as np

In [4]:
df= pd.read_csv(r'C:\Users\Nikhil\Desktop\web dev\house price prediction\data.csv')

In [5]:
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [6]:
df=df.drop(['date','waterfront','view','condition','street','country','statezip'],axis=1)

In [7]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,city
0,313000.0,3.0,1.50,1340,7912,1.5,1340,0,1955,2005,Shoreline
1,2384000.0,5.0,2.50,3650,9050,2.0,3370,280,1921,0,Seattle
2,342000.0,3.0,2.00,1930,11947,1.0,1930,0,1966,0,Kent
3,420000.0,3.0,2.25,2000,8030,1.0,1000,1000,1963,0,Bellevue
4,550000.0,4.0,2.50,1940,10500,1.0,1140,800,1976,1992,Redmond


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_set,test_set=train_test_split(df,test_size=0.2,random_state=42)

In [10]:
train_set.shape

(3680, 11)

In [11]:
test_set.shape

(920, 11)

In [12]:
target_column_name="price"

In [13]:
input_feature_train_df=train_set.drop(columns=[target_column_name],axis=1)
target_feature_train_df=train_set[target_column_name]
input_feature_test_df=test_set.drop(columns=[target_column_name],axis=1)
target_feature_test_df=test_set[target_column_name]

In [14]:
print(f"Shape of input_feature_train_arr: {input_feature_train_df.shape}")
print(f"Shape of target_feature_train_df: {target_feature_train_df.shape}")
print(f"Shape of input_feature_test_arr: {input_feature_test_df.shape}")
print(f"Shape of target_feature_test_df: {target_feature_test_df.shape}")


Shape of input_feature_train_arr: (3680, 10)
Shape of target_feature_train_df: (3680,)
Shape of input_feature_test_arr: (920, 10)
Shape of target_feature_test_df: (920,)


In [16]:
import os
import sys
from src.exception import CustomException
from src.logger import logging
from src.components.data_transformation import DataTransformation
from src.components.data_transformation import DataTransformationConfig
import pandas as pd

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler

from src.exception import CustomException
from src.logger import logging
import os

In [17]:
class DataTransformationConfig:
    preprocessor_obj_file_path=os.path.join('artifacts',"proprocessor.pkl")

class DataTransformation:
    def __init__(self):
        self.data_transformation_config=DataTransformationConfig()

    def get_data_transformer_object(self):
        '''
        This function si responsible for data trnasformation
        
        '''
        try:
            categorical_columns=['city']
            numerical_columns=['bedrooms','bathrooms','sqft_living','sqft_lot','floors','sqft_above','sqft_basement','yr_built','yr_renovated']

            num_pipeline= Pipeline(
                steps=[
                ("imputer",SimpleImputer(strategy="median")),
                ("scaler",StandardScaler())

                ]
            )

            cat_pipeline=Pipeline(

                steps=[
                ("imputer",SimpleImputer(strategy="most_frequent")),
                ("one_hot_encoder",OneHotEncoder(handle_unknown='ignore')),
                ("scaler",StandardScaler(with_mean=False))
                ]

            )

            logging.info(f"Categorical columns: {categorical_columns}")
            logging.info(f"Numerical columns: {numerical_columns}")

            preprocessor=ColumnTransformer(
                [
                ("num_pipeline",num_pipeline,numerical_columns),
                ("cat_pipelines",cat_pipeline,categorical_columns)

                ]


            )

            return preprocessor
        
        except Exception as e:
            raise CustomException(e,sys)

In [18]:
preprocessing_obj=DataTransformation()

In [19]:
preprocessing_obj=preprocessing_obj.get_data_transformer_object()

In [20]:
input_feature_train_arr=preprocessing_obj.fit_transform(input_feature_train_df)
input_feature_test_arr=preprocessing_obj.transform(input_feature_test_df)

In [21]:
input_feature_train_arr.shape

(3680, 52)

In [22]:
type(input_feature_train_arr)

scipy.sparse._csr.csr_matrix

In [23]:
type(target_feature_train_df)

pandas.core.series.Series

In [25]:
train_arr = np.c_[
    input_feature_train_arr, np.array(target_feature_train_df).reshape(-1,1)
]

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 1 has size 3680